<strong><h1>Criteria Point System</h1></strong>


</h2></strong>Pathogenic Significance</h2></strong> 

In [ ]:
########################################################### Algorithm new PS #######################################################################
import pandas as pd

excel_file_path = r'C:\Users\hayae\Desktop\New_opencravat_summary_onepage.xlsx'
#sheet_name = 'PR_001_new'

df = pd.read_excel(excel_file_path)

df['Points_PS'] = 0

# Define lists of genes worth 2 Points_PS and 1 point each
genes_worth_2_Points_PS = ['BRCA2', 'BRIP1', 'ATM', 'BARD1', 'BRCA1', 'CDK12', 'CHEK1', 'CHEK2', 'FANCL', 'PALB2', 'PPP2R2A', 'RAD51B', 'RAD51C',
                        'RAD51D', 'RAD54L']  # Add more genes as needed
df['Points_PS'] += df['Gene'].apply(lambda x: 2 if x in genes_worth_2_Points_PS else 0)

genes_worth_1_point = ['APC', 'AR', 'ARID1A', 'FOXA1', 'MYC', 'PTEN', 'SPOP', 'RB1', 'TP53', 'TMPRSS2', 'KMT2B', 'KMT2C', 'KMT2D', 'ZFHX3',
                       'CTNNB1', 'PIK3CA', 'KDM6A', 'GRIN2A', 'MED12', 'PTPRD', 'SPEN', 'MGA', 'PIK3R1', 'PIK3R2', 'NCOR1', 'ASXL1', 'JAK1',
                       'NF1', 'ARID1A', 'ASXL2', 'EPHB1', 'TBX3', 'FAT1', 'RYBP', 'ROS1', 'NOTCH4', 'SETD2', 'SF3B1']  # Add more genes as needed
df['Points_PS'] += df['Gene'].apply(lambda x: 1 if x in genes_worth_1_point else 0)

# Check if REVEL_Score is available
if 'REVEL_Score' in df.columns:
    # If REVEL_Score is available, assign Points_PS based on its value
    df['Points_PS'] += (df['REVEL_Score'] > 0.7).astype(int) * 2  # Add 2 Points_PS if REVEL_Score > 0.7
    df['Points_PS'] += (df['REVEL_Score'] < 0.7).astype(int) * -2  # Deduct 2 Points_PS if REVEL_Score < 0.7
else:
    # If REVEL_Score is not available, use CADD_Phred to assign Points_PS
    df['Points_PS'] += (df['CADD_Phred'] > 15).astype(int) * 2  # Add 2 Points_PS if CADD_Phred > 15
    df['Points_PS'] += (df['CADD_Phred'] < 15).astype(int) * -2  # Deduct 2 Points_PS if CADD_Phred < 15

# Fill NaN values with a placeholder value (e.g., 'Unknown')
df['ClinVar_Clinical_Significance'] = df['ClinVar_Clinical_Significance'].fillna('Unknown')

# Assign Points_PS based on ClinVar_Clinical_Significance
df['Points_PS'] += df['ClinVar_Clinical_Significance'].str.contains('Benign').astype(int) * -5  
df['Points_PS'] += df['ClinVar_Clinical_Significance'].str.contains('Conflicting interpretations of pathogenicity').astype(int) * 3.5  
df['Points_PS'] += df['ClinVar_Clinical_Significance'].str.contains('Uncertain significance').astype(int) * 2.5  
df['Points_PS'] += df['ClinVar_Clinical_Significance'].str.contains('Conflicting classifications of pathogenicity').astype(int) * 3.5  
df['Points_PS'] += df['ClinVar_Clinical_Significance'].str.contains('Pathogenic').astype(int) * 5  

# Assign Points_PS based on COSMIC_Gene_Tissue_count
df['Points_PS'] += df['COSMIC_Variant_Count'].apply(lambda x: 1 if isinstance(x, str) and x.lower().startswith('prostate') and int(x.split(',')[1].strip(']').strip()) <= 9 else (3 if isinstance(x, str) and int(x.split(',')[1].strip(']').strip()) >= 10 else 0))

# Add Points_PS based on MutPred_Score (after converting to numeric)
df['Points_PS'] += (df['MutPred_Score'] > 0.5).fillna(False).astype(int) * 2  # Add 2 Points_PS if MutPred_Score > 0.5
df['Points_PS'] += (df['MutPred_Score'] < 0.5).fillna(False).astype(int) * -2

# Assign Points_PS based on 'Cancer_Gene Census_Driver_Class' column
df['Points_PS'] += df['Cancer_Gene Census_Driver_class'].apply(lambda x: 3 if isinstance(x, str) and (('Oncogene' in x.lower()) or ('TSG' in x.lower())) else 0)

# Check if REVEL_Score is available
if 'Cancer_Gene Census_Tumor_types somatic' in df.columns:
    df['Points_PS'] += df['Cancer_Gene Census_Tumor_types somatic'].apply(lambda x: 3 if isinstance(x, str) and 'prostate' in x.lower() else 0)
else:
    df['Points_PS'] += df['Cancer_Gene Census_Tumor_types germline'].apply(lambda x: 3 if isinstance(x, str) and 'prostate' in x.lower() else 0)

# Assign 2 Points_PS if 'oncogene', 'TSG', or 'fusion' occur in the 'Cancer_Hotspots_Cancer_Types' column
df['Points_PS'] += df['Cancer_Hotspots_Cancer_Types'].apply(lambda x: 3 if isinstance(x, str) and (('Oncogene' in x.lower()) or ('TSG' in x.lower())) else 0) 

# Add Points_PS based on PhD-SNPg_Score (after converting to numeric)
df['Points_PS'] += (df['PhD-SNPg_Score'] > 0.5).fillna(False).astype(int) * 2  # Add 2 Points_PS if PhD-SNPg_Score > 0.5
df['Points_PS'] += (df['PhD-SNPg_Score'] < 0.5).fillna(False).astype(int) * -2

# Add Points_PS based on gnomAD_Gene_LoF_Z-Score (after converting to numeric)
df['Points_PS'] += (df['gnomAD_Gene_LoF_Z-Score'] > 3).fillna(False).astype(int) * 5  # Add 5 Points_PS if gnomAD_Gene_LoF_Z-Score > 3
df['Points_PS'] += ((df['gnomAD_Gene_LoF_Z-Score'] <= 3) & (df['gnomAD_Gene_LoF_Z-Score'] > 2)).fillna(False).astype(int) * 3  # Add 3 Points_PS if 2 < gnomAD_Gene_LoF_Z-Score <= 3
df['Points_PS'] += (df['gnomAD_Gene_LoF_Z-Score'] <= 2).fillna(False).astype(int) * -5  # Add 1 point if gnomAD_Gene_LoF_Z-Score <= 2

# Convert 'LoFtool_LoF_Score' to numeric, coercing errors to NaN
df['LoFtool_LoF_Score'] = pd.to_numeric(df['LoFtool_LoF_Score'], errors='coerce')

# Add Points_PS based on LoFtool_LoF_Score
df['Points_PS'] += (df['LoFtool_LoF_Score'] > 0.5).fillna(False).astype(int) * 2  # Add 2 Points_PS if LoFtool_LoF_Score > 0.5
df['Points_PS'] += (df['LoFtool_LoF_Score'] < 0.5).fillna(False).astype(int) * -2  # Deduct 2 Points_PS if LoFtool_LoF_Score < 0.5


print(df['Points_PS'])

output_excel_file_path = r'C:\Users\hayae\Desktop\OutputPSnew.xlsx'

df.to_excel(output_excel_file_path, index=False)

print("Results exported to Excel successfully!")



In [ ]:
# Add a new column 'Pathogenicity' based on the points
df['Pathogenicity Prediction'] = pd.cut(df['Points_PS'], bins=[-float('inf'), 0, 6, 12, 18, float('inf')],
                              labels=['Benign', 'Likely Benign', 'VUS', 
                                      'Likely Pathogenic', 'Pathogenic'], include_lowest=True)

output_excel_file_path_with_prediction = r'C:\Users\hayae\Desktop\all_output_with_prediction.xlsx'
df[['Points_PS', 'Pathogenicity Prediction']].to_excel(output_excel_file_path_with_prediction, index=False)
print("Results with Pathogenicity Prediction exported to Excel successfully!")



</h2></strong>Clinical Significance</h2></strong> 

In [ ]:
########################################################### Algorithm new CS #######################################################################

import pandas as pd

excel_file_path = r'C:\Users\hayae\Desktop\New_opencravat_summary_onepage.xlsx'
df = pd.read_excel(excel_file_path)

df['CS Points'] = 0

# Define lists of genes worth 2 points and 1 point each
genes_worth_2_points = ['BRCA2', 'BRIP1', 'ATM', 'BARD1', 'BRCA1', 'CDK12', 'CHEK1', 'CHEK2', 'FANCL', 'PALB2', 'PPP2R2A', 'RAD51B', 'RAD51C',
                        'RAD51D', 'RAD54L']  # Add more genes as needed
df['CS Points'] += df['Gene'].apply(lambda x: 2 if x in genes_worth_2_points else 0)

genes_worth_1_point = ['APC', 'AR', 'ARID1A', 'FOXA1', 'MYC', 'PTEN', 'SPOP', 'RB1', 'TP53', 'TMPRSS2', 'KMT2B', 'KMT2C', 'KMT2D', 'ZFHX3',
                       'CTNNB1', 'PIK3CA', 'KDM6A', 'GRIN2A', 'MED12', 'PTPRD', 'SPEN', 'MGA', 'PIK3R1', 'PIK3R2', 'NCOR1', 'ASXL1', 'JAK1',
                       'NF1', 'ARID1A', 'ASXL2', 'EPHB1', 'TBX3', 'FAT1', 'RYBP', 'ROS1', 'NOTCH4', 'SETD2', 'SF3B1']  # Add more genes as needed
df['CS Points'] += df['Gene'].apply(lambda x: 1 if x in genes_worth_1_point else 0)

# Assign points based on DGIdb:_The Drug Interaction Database_Drug_Name
drug_names = ['Niraparib Tosylate Monohydrate', 'Abiraterone Acetate', 'Olaparib', 'Rucaparib Camsylate']
df['CS Points'] += df['DGIdb:_The Drug Interaction Database_Drug_Name'].astype(str).apply(lambda x: 5 if any(name in x for name in drug_names) else 0)

# Assign points based on DGIdb:_The Drug Interaction Database_All_Annotations
drug_names = ['OLAPARIB','NIRAPARIB', 'ABIRATERONE', 'RUCAPARIB']
df['CS Points'] += df['DGIdb:_The Drug Interaction Database_All_Annotations'].astype(str).apply(lambda x: 5 if any(name in x for name in drug_names) else 0)

# CIVIC Criteria 

#EL - A
genes_worth_3_points = ['PALB2', 'ATM', 'BRCA1', 'CDK12', 'CHEK2', 'RAD54L', 'BRIP1', 'BARD1', 'CHEK1', 'RAD51B', 'RAD51D', 'FANCL', 'RAD51C', 'BRCA2']
df['CS Points'] += df['Gene'].apply(lambda x: 3 if x in genes_worth_3_points else 0)

#EL - B

genes_civic_worth_2_points = ['PPP2RA', 'PTEN']
df['CS Points'] += df['Gene'].apply(lambda x: 3 if x in genes_civic_worth_2_points else 0)

# Add 2 points if 'HOXB13' gene and 'p.Gly84Glu' protein change are found in the same row
df['CS Points'] += df.apply(lambda row: 2 if row['Gene'] == 'HOXB13' and row['Protein_Change'] == 'p.Gly84Glu' else 0, axis=1)
df['CS Points'] += df.apply(lambda row: 2 if row['Gene'] == 'TSPYL1' and row['Protein_Change'] == 'p.Pro62Ser' else 0, axis=1)
df['CS Points'] += df.apply(lambda row: 2 if row['Gene'] == 'CHEK2' and row['Protein_Change'] == 'p.Ile157Thr' else 0, axis=1)
df['CS Points'] += df.apply(lambda row: 2 if row['Gene'] == 'MRE11' and row['Sequence_Ontology'] in ['frameshift_truncation', 'frameshift_elongation'] else 0, axis=1)

#EL - C

df['CS Points'] += df.apply(lambda row: 1 if row['Gene'] == 'FANCA' and row['Protein_Change'] == 'p.Ser1088Phe' else 0, axis=1)

#EL - D

df['CS Points'] += df.apply(lambda row: 1 if row['Gene'] == 'AR' and row['Protein_Change'] == 'p.Try724' else 0, axis=1)

print(df['CS Points'].head(20))


output_excel_file_path = r'C:\Users\hayae\Desktop\CS_Points.xlsx'

# Write the DataFrame with CS Points to an Excel file
df.to_excel(output_excel_file_path, index=False)

print("Results exported to Excel successfully!")


In [ ]:
df['Clinical Significance'] = pd.cut(df['CS Points'], bins=[-float('inf'), 4, float('inf')],
                              labels=['Not Clinically Significant', 'Clinically Significant'], include_lowest=True)

excel_file_path = r'C:\Users\hayae\Desktop\cs_points.xlsx'
df.to_excel(excel_file_path, index=False)

print(df[['CS Points', 'Clinical Significance']])
